In [1]:
%pip install llmsherpa langchain chromadb
!apt-get install git-lfs
!git lfs install

ERROR! Session/line number was not unique in database. History logging moved to new session 2
Note: you may need to restart the kernel to use updated packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 64 not upgraded.
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


In [2]:
!git config --global credential.helper 'cache --timeout=3600'
!echo "machine huggingface.co login user password $token" > ~/.netrc

In [ ]:
import getpass
token = getpass.getpass('HuggingFace Token')

HuggingFace Token ·····································


In [ ]:
%pip install chromadb

In [4]:
%pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.7 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=emb_model,
    cache_folder=os.getenv('SENTENCE_TRANSFORMERS_HOME')
)
# The vectorstore to use to index the summaries
vectorstore = Chroma(
    collection_name="mm_rag_mistral",
    embedding_function=embeddings,
    persist_directory="FiscaBOT_vector_store",
)

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
import uuid
from langchain.schema.document import Document

 # Initialize the storage layer
store = InMemoryStore()
id_key = "doc_id"

retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key
    )


def add_documents(retriever, doc_contents, filename, document_type):
    doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
    child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
    sub_docs = []

    # Assuming the first two contents are company name and sector
    # company_name = doc_contents[0] if len(doc_contents) > 0 else "Unknown"
    # activity_sector = doc_contents[1] if len(doc_contents) > 1 else "Unknown"
    # document_context = f"Company: {company_name}, Sector: {activity_sector}"

    for i, doc in enumerate(doc_contents):
        _id = doc_ids[i]
        _sub_docs = child_text_splitter.create_documents([doc])
        for _doc in _sub_docs:
            _doc.metadata['doc_id'] = _id
            _doc.metadata['pdf_name'] = filename
#             _doc.metadata['year'] = year
            _doc.metadata['document_type'] = document_type
            # _doc.metadata['document_context'] = document_context  # New metadata field

        sub_docs.extend(_sub_docs)

    retriever.vectorstore.add_documents(sub_docs)
    retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

In [12]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)

In [13]:
# Categorize elements by type
def categorize_elements(doc):
    """
    Categorize extracted elements from a PDF into  texts.
    """
    texts=[]

    for chunk in doc.chunks():
      texts.append(str(chunk.to_context_text()))

    return texts

In [14]:
import os
import concurrent.futures

def process_pdf(folder_path, filename):
    pdf_path = os.path.join(folder_path, filename)
    doc = pdf_reader.read_pdf(pdf_path)  # Assuming 'read_pdf' is the correct method

    # Obtenir le nom du dossier parent comme type de document
    document_type = os.path.basename(os.path.dirname(pdf_path))

    add_documents(retriever, categorize_elements(doc), filename, document_type)
    return f"Processed {filename}"

def main(root_folder):
    # Créer un ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        # Parcourir récursivement tous les répertoires
        for folder_path, _, filenames in os.walk(root_folder):
            for filename in filenames:
                if filename.endswith(".txt"):
                    # Soumettre des tâches à l'executor
                    future = executor.submit(process_pdf, folder_path, filename)
                    futures.append(future)
        
        # Gérer les résultats dès qu'ils sont disponibles
        for future in concurrent.futures.as_completed(futures):
            try:
                print(future.result())
            except Exception as e:
                print(f"Une erreur s'est produite : {e}")

if __name__ == "__main__":
    main("/kaggle/input/datasetpdf-fiscabot")
    vectorstore.persist()


In [17]:
!zip -r FiscaBOT_vector_store.zip FiscaBOT_vector_store


  adding: FiscaBOT_vector_store/ (stored 0%)
  adding: FiscaBOT_vector_store/chroma.sqlite3 (deflated 98%)


In [18]:
import os

# Specify the path to the zip file
zip_file_path = 'FiscaBOT_vector_store'

# Get the size of the zip file
zip_file_size = os.path.getsize(zip_file_path)

# Convert size to human-readable format
def convert_bytes(size):
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return "%3.1f %s" % (size, x)
        size /= 1024.0

# Print the size of the zip file
print("Size of the zip file:", convert_bytes(zip_file_size))


Size of the zip file: 4.0 KB
